### 1.1 优先级队列

#### 一. 何为优先级队列
1. 此前的搜索树,词典结构都支持数据全集的访问和操作. 而优先级队列, 不保证全序, 仅保证偏序.仅将操作对象限制在当前的全局极值者  
2. 从数据结构的角度看, 无论是排序的节点是数值,超字符权重,还是事件发生的时间,数据项之间都可以比较大小, 这种大小关系称为优先级  
 也因此, 优先级队列和词典实现的dictionary区别在于, 关键码在词典中不用有大小关系, 只有判等关系即可. 而优先级队列需要关键码的大小比较  
3. 优先级队列插入,查找,删除均可以在$O(logn)$内完成, 其中查找(当前最大值/最小值)只需要$O(1)$  

#### 二. 堆
1. 偏序关系可以依靠堆(heap)维护

### 1.2 完全二叉堆

#### 一. 完全二叉堆的结构  
1. 完全二叉堆的逻辑结构等同于完全二叉树  
2. 完全二叉堆的优先级满足 :  
 堆顶以外的每个节点, 其优先级都不大于其父节点优先级, 此即所谓的"堆序性"  
3. 大顶堆与小顶堆  
 大顶堆是堆顶元素优先级最高的堆,小顶堆是堆顶优先级最小的堆. 因此, 堆的`getMax()`操作总是在$O(1)$时间内返回  
 
#### 二. 完全二叉堆的向量表示
1. 尽管线性二叉树不能用向量表示, 但是完全二叉树可以使用向量表示, 所以对应的完全二叉堆也可以用向量表示.  
 而其表示方法为:若完全二叉堆的规模为n,则按照层次遍历的次序,每个节点都应该对应于唯一的编号. 所以若将所有节点组织为一个向量, 则堆中各节点的编号与各单元的秩(`index`)也一一对应  
<img src='img/wanquanerchadui1.png' height='70%' width='70%'>  
 
2. 二叉关系的建立, 纯粹是根据向量的角标计算而来.   
 具体的, 完全二叉堆中任意节点$v$必满足(根节点从index=0开始,这也和向量从0开始暗合):  
  1. 若$v$有左孩子, 则 $i(v.lc)=2*i(v)+1$  
  2. 若$v$有右孩子, 则 $i(v.lc)=2*i(v)+2$
  3. 若$v$有父节点, 则 $i(v.parent) = \left\lfloor \frac { i(v)-1 }{ 2 }  \right\rfloor =\left\lceil \frac { i(v) }{ 2 } -1 \right\rceil $  
  
3. 如下`ComplHeapUtil`封装了堆中节点的操作

In [1]:
class ComplHeapUtil(object):
    @staticmethod
    def inheap(n,i):
        '''判断pq[i]是否合理,n为节点个数'''
        return (i>-1) and (i<n)
    @staticmethod
    def parent(i):
        '''找到节点i的父节点'''
        return (i-1)/2
    @staticmethod
    def last_internal(n):
        '''优先队列的最后一个内部节点(即末节点的父节点)'''
        return ComplHeapUtil.parent(n-1)
    @staticmethod
    def lchild(i):
        '''节点的左孩子'''
        return i*2 + 1
    @staticmethod
    def rchild(i):
        '''节点右孩子'''
        return i*2 + 2
    @staticmethod
    def parent_valid(i):
        '''判断节点i是否有父节点(只要节点不是根节点, 就有父节点)'''
        return i>0
    @staticmethod
    def lchild_valid(n,i):
        '''判断节点i是否有左孩子'''
        return ComplHeapUtil.inheap(n,ComplHeapUtil.lchild(i))
    @staticmethod
    def rchild_valid(n,i):
        '''判断节点i是否有右孩子'''
        return ComplHeapUtil.inheap(n,ComplHeapUtil.rchild(i))
    @staticmethod
    def bigger(pq,i,j):
        '''取节点i和j中较大的节点'''
        return j if pq[j]>pq[i] else i
    @staticmethod
    def proper_parent(elem,n,i):
        '''选择父节点和2个孩子节点中最大的那个(最多2个孩子),作为调整堆时上滤和下滤的依据'''
        if ComplHeapUtil.rchild_valid(n,i):
            bigger1 = ComplHeapUtil.bigger(elem,i,ComplHeapUtil.lchild(i))
            biggger2 = ComplHeapUtil.bigger(elem,bigger1,ComplHeapUtil.rchild(i))
            return biggger2
        else:
            if ComplHeapUtil.lchild_valid(n,i): #没有右孩子但有左孩子
                return ComplHeapUtil.bigger(elem,i,ComplHeapUtil.lchild(i))
            else:    #没有右孩子也没有左孩子
                return i

#### 三. 大顶堆的插入,删除,查找,建堆
1. 查找  
 堆只能操作极值点`getMax`, 即直接返回堆顶
2. 插入  
 插入分为两个步骤  
  1. 将新词条接至向量末尾  
  2. 对该词条执行上滤  
   所谓上滤是指, 新插入节点后可能会破坏父节点优先级大于孩子节点的约定. 因此需要调整, 只要发现新节点比父节点的优先级大, 就交换父节点和新插入节点, 并一层一层的向上比较, 直到彻底满足约定.时间复杂度和节点深度有关,为$O(logn)$
3. 删除  
 删除也分为2步  
  1. 摘除堆顶元素,并使用末尾元素占用堆顶位置    
  2. 对新的堆顶进行下滤  
   所谓下滤, 和上滤意义一样, 都是为了维持父节点优先级大于孩子节点的约定.   
    但要注意, 下滤时节点要和其2个孩子节点作比较, 并选择二者中最大的那个孩子节点进行交换  
    其时间复杂度和堆顶的高度有关, 同样为$O(logn)$
    
#### 四. 建堆
1. 我们首先想到的思路, 是自顶向下逐一的调用`insert`方法, 从而构建好堆.这种方法的一个弊端是每个节点都要经过一轮上滤.随着树深度的增加, 上滤交换的次数也会线性的增大.我们转而考虑另一种方法:  
  1. 首先使用Vector顺序的构建一个完全二叉树  
  2. 随后对树的内部节点(不包含叶子节点), 从底向上对每个内部节点执行下滤,从而调整好大顶堆  
2. 上述建堆的方法称为弗洛伊德算法.  
 表面上看, 弗洛伊德和蛮力法调用`insert`的区别在于, 弗洛伊德是自下向上下滤, 蛮力法是自上而下的上滤.但弗洛伊德建堆真正快速的原因是, 完全二叉树中, 深度大的节点远远大于高度大的节点

In [2]:
class PQ_ComplHeap(object):
    def __init__(self):
        self.__elem = []  #堆对应的向量
        self.__size = 0

    def getMax(self):
        '''查找堆顶'''
        return self.__elem[0]

    def insert(self,data):
        self.__elem.append(data) #插入向量末尾
        self.__size = self.__size+1
        self.__percolate_up(self.__size-1) #将新插入的处在最后的词条上滤


    def delMax(self):
        max_val = self.__elem[0] #记录当前堆顶
        self.__elem[0] = self.__elem[self.__size-1] #将最后一个叶子节点挪到堆顶
        self.__elem = self.__elem[:-1]
        self.__size = self.__size-1
        self.__percolate_down(0,self.__size)
        return max_val

    def __percolate_up(self,i):
        '''节点上滤'''
        while ComplHeapUtil.parent_valid(i):
            parent = ComplHeapUtil.parent(i)
            if self.__elem[i]>self.__elem[parent]: #孩子大于父亲,交换
                tmp = self.__elem[parent]
                self.__elem[parent] = self.__elem[i]
                self.__elem[i] = self.__elem[parent]
                i = parent
            else:
                break
        return i

    def __percolate_down(self,i,n):
        '''节点下滤'''
        while 1:
            biggest = ComplHeapUtil.proper_parent(self.__elem,n,i)
            if i!=biggest: #当前节点比孩子优先级小的情况
                tmp = self.__elem[i]
                self.__elem[i] = self.__elem[biggest]
                self.__elem[biggest] = tmp
                #交换后向下层递归
                i = biggest
            else:
                break
        return i

    def heapify(self,vector):
        '''弗洛伊德建堆: 自底向上执行下滤'''
        self.__elem = vector
        self.__size = len(vector)
        n = len(self.__elem)
        last_internal = ComplHeapUtil.last_internal(n)
        for i in range(last_internal,-1,-1):
            self.__percolate_down(i,n)

    def getElem(self):
        return self.__elem

In [3]:
if __name__ == "__main__":
    heap = PQ_ComplHeap()
    heap.heapify([3,2,5,6,9,7,10])
    print heap.getMax()
    print heap.getElem()
    heap.delMax()
    print heap.getElem()
    heap.delMax()
    print heap.getElem()

10
[10, 9, 7, 6, 2, 3, 5]
[9, 6, 7, 5, 2, 3]
[7, 6, 3, 5, 2]


#### 五. 使用完全二叉堆进行堆排序
1. 首先使用弗洛伊德建堆  
2. 其次, 每次调用delMax选取当前的最大值

In [4]:
class Heap_sort(PQ_ComplHeap):
    def sort(self,vector):
        self.heapify(vector)
        n = len(vector)
        new_vector = []
        for i in range(n):
            new_vector.append(self.delMax())
        return new_vector

In [5]:
if __name__ == "__main__":
    h = Heap_sort()
    print h.sort([99,1,4,2,8,6])

[99, 8, 6, 4, 2, 1]
